# SenasJanAI Transformerモデル トレーニングハブ

## 1. 環境設定

In [2]:
!pip uninstall mahjong -y
!pip install mahjong==1.3.0

from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/MyDrive/いろいろ/麻雀AI/googlecolab/senasJanAI"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 3.2 MB/s eta 0:00:00
Mounted at /content/drive
/content/drive/MyDrive/いろいろ/麻雀AI/googlecolab/senasJanAI


## 2. データセットの準備
Google Driveから `data.zip` をColabの高速ローカルストレージに展開します。

In [3]:
import os
import zipfile

ZIP_FILE_PATH = 'data.zip'
EXTRACT_DIR = '/content/data'

print(f"Copying '{ZIP_FILE_PATH}' to local environment...")
!cp -f "{ZIP_FILE_PATH}" /content/

print(f"Unzipping files to '{EXTRACT_DIR}'...")
if os.path.isdir(EXTRACT_DIR):
    !rm -rf {EXTRACT_DIR}
os.makedirs(EXTRACT_DIR, exist_ok=True)

with zipfile.ZipFile('/content/data.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

print("Dataset prepared successfully.")

Copying 'data.zip' to local environment...
Unzipping files to '/content/data'...
Dataset prepared successfully.


## 3. データ生成（チャンク処理）
ローカルの牌譜ログをチャンク（小さな塊）に分けて処理し、メモリ不足を完全に防ぎます。
処理完了後、自動的に一つのデータセットに結合されます。

In [ ]:
# sedコマンドは不要になりました
!python -m src.transformer.generate_data

In [5]:
# ===============================================================
#  3.5. デバッグ：1ファイル分のデータ生成結果を確認
# ===============================================================
# 1ファイルだけを処理して、生成されるデータの中身を確認します。
# これを実行しても、全体のデータセット(pklファイル)は作成されません。
# ===============================================================

!python -m src.transformer.debug_parser

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/content/drive/MyDrive/いろいろ/麻雀AI/googlecolab/senasJanAI/src/transformer/debug_parser.py", line 8, in <module>
    from .transformer_parser import TransformerParser
ImportError: cannot import name 'TransformerParser' from 'src.transformer.transformer_parser' (/content/drive/MyDrive/いろいろ/麻雀AI/googlecolab/senasJanAI/src/transformer/transformer_parser.py)


## 4. モデルの学習

In [ ]:
!python -m src.transformer.train_transformer

## 5. 予測の実行

In [ ]:
!python -m src.transformer.predict_transformer